Etapas procesamiento:
- Molienda: 4 equipos cada uno con 3000 kg/hr y merma 0.05
- Prensado: 6 equipos cada uno con 2500 kg/hr y merma 0.15
- Clarificacion: 2 equipos cada uno con 5000 kg/hr y merma 0.35

**SIMULACION**

In [4]:
import math

In [ ]:
#fucnion que convierte el diccionario de los lotes que compro en un diccionario de camiones que llegan
#"CamionX"=[tipo_uva,kg,calidad,tiempo_llegada]
def generar_camiones(dicc_lotes):
    dicc_camiones={}
    j=1
    for i in dicc_lotes.values():
        tipo_uva=i[0]
        kg=i[1]
        calidad=i[2]
        tiempo_llegada=i[3]
        tsmolienda=0
        tsprensado=0
        tsclarificacion=0
        dicc_camiones["CAMION{}".format(j)]=[tipo_uva,kg,calidad,tiempo_llegada,tsmolienda,tsprensado,tsclarificacion]
        j+=1
    return dicc_camiones

In [14]:
#funcion que genera diccionarios de los tiempos de salida de los equipos en cada etapa
#y diccionarios de los estados de los equipos en cada etapa
def generar_molienda(equipos):
    tiempos_molienda={}
    estados_molienda={}
    for i in range(equipos):
        tiempos_molienda["TSM{}".format(i+1)]=math.inf
        estados_molienda["M{}".format(i+1)]=[0,0]
    return tiempos_molienda,estados_molienda
def generar_prensado(equipos):
    tiempos_prensado={}
    estados_prensado={}
    for i in range(equipos):
        tiempos_prensado["TSP{}".format(i+1)]=math.inf
        estados_prensado["P{}".format(i+1)]=[0,0]
    return tiempos_prensado,estados_prensado
def generar_clarificacion(equipos):
    tiempos_clarificacion={}
    estados_clarificacion={}
    for i in range(equipos):
        tiempos_clarificacion["TSC{}".format(i+1)]=math.inf
        estados_clarificacion["C{}".format(i+1)]=[0,0]
    return tiempos_clarificacion,estados_clarificacion

In [ ]:
#funcion que decide el prox evento a ocurrir
def prox_evento(dicc_camiones,tiempos_molienda,tiempos_prensado,tiempos_clarificacion):
    tiempos=[]
    for i in dicc_camiones:
        tiempos.append([i,dicc_camiones[i][3]])
    for i in tiempos_molienda:
        tiempos.append([i,tiempos_molienda[i]])
    for i in tiempos_prensado:
        tiempos.append([i,tiempos_prensado[i]])
    for i in tiempos_clarificacion:
        tiempos.append([i,tiempos_clarificacion[i]])
    tipo_evento=math.inf
    tiempo_evento=math.inf
    for i in tiempos:
        if i[1]<tiempo_evento:
            tipo_evento=i[0]
            tiempo_evento=i[1]
    return tipo_evento,tiempo_evento

In [ ]:
def llegada(T,tipo_evento,dicc_camiones,tiempos_molienda,estados_molienda,tasa_molienda,COLA):
    #rescato info del camion
    #si es una llegada, tipo_evento sera el id de un camion
    tipo_uva=dicc_camiones[tipo_evento][0]
    kg=dicc_camiones[tipo_evento][1]
    calidad=dicc_camiones[tipo_evento][2]
    tiempo_fuera=dicc_camiones[tipo_evento][3]
    dicc_camiones[tipo_evento][3]=math.inf #actualizo el tiempo de llegada del camion a infinito
    print("----> Llega el camion {} a las {} horas con tipo uva {}, {} kg, calidad {}.".format(tipo_evento,tiempo_evento,tipo_uva,kg,calidad))
    #lo asigno a algun equipo molienda
    equipo_asignado=0
    #reviso los equipos de molienda y si encuentro uno desocupado se lo asigno
    for i in estados_molienda:
        if estados_molienda[i][0]==0:
            equipo_asignado=i
            estados_molienda[i]=[1,tipo_evento]
            tiempo_salida=(kg/tasa_molienda)
            tiempos_molienda["TSM{}".format(equipo_asignado[1])]=T+tiempo_salida
            dicc_camiones[tipo_evento][4]=tiempo_salida #tiempo que demora la molienda de este camion
            print("Se asigna al equipo {} de molienda. Saldrá en {} horas más.".format(equipo_asignado,tiempo_salida))
            break
    #si estan todos ocupados lo asigno a la cola
    if equipo_asignado==0:
        print("Todos los equipos de molienda estan ocupados, se agrega a la cola a las {} horas".format(T))
        COLA.append([tipo_evento,T])

In [ ]:
def salida_molienda(T,tipo_evento,dicc_camiones,tiempos_molienda,estados_molienda,tasa_molienda,merma,tiempos_prensado,estados_prensado,tasa_prensado,COLA):
    #si es salida, tipo de evento sera TSMx Ej "TSM1"
    #rescato y actualizo info del camion
    camion=estados_molienda[tipo_evento[1:]][1] #camion que esta saliendo de molienda Ej "CAMION1"
    tipo_uva=dicc_camiones[camion][0]
    dicc_camiones[camion][1]=dicc_camiones[camion][1]*(1-merma) #merma que se produce en molienda
    kg=dicc_camiones[camion][1]
    calidad=dicc_camiones[camion][2]
    dicc_camiones[camion][3]=dicc_camiones[camion][3]+T #contador del tiempo que lleva fuera la uva
    #ver si prensado esta desocupado
    equipo_asignado=0 
    for i in estados_prensado:
        if estados_prensado[i][0]==0:
            equipo_asignado=i #Ej "M1"
            estados_prensado[i]=[1,camion] #Ej [1,"CAMION1"]
            tiempo_salida=(kg/tasa_prensado)
            tiempos_prensado["TSP{}".format(equipo_asignado[1])]=T+tiempo_salida
            dicc_camiones[camion][5]=tiempo_salida
            print("Se asigna al equipo {} de prensado. Saldrá en {} horas más.".format(equipo_asignado,tiempo_salida))
            break
    #si no hay ningun equipo prensado desocupado
    if equipo_asignado==0:
        estados_molienda[tipo_evento[1:]][0]=2 
        tiempos_molienda[tipo_evento]=math.inf
        print("No hay ningun equipo de prensado desocupado. Se bloquea")
    #revisar cola
    else:
        if len(COLA)==0: #si no hay camiones en cola
            estados_molienda[tipo_evento[1:]]=[0,0] #Ej "M1"=[0,0]
            tiempos_molienda[tipo_evento]=math.inf #Ej "TSM1"=inf
            print("No hay camiones esperando en cola")
        else: #si hay camiones en cola entra el primero
            camion=COLA[0][0] #Ej "CAMION 1"
            estados_molienda[tipo_evento[1:]]=[1,camion] #Ej "M1"=[1,"CAMION 1"]
            equipo=estados_molienda[tipo_evento[1:]]
            kg=dicc_camiones[camion][1]
            tiempo_salida=T+(kg/tasa_molienda)
            tiempos_molienda[tipo_evento]=T+tiempo_salida
            dicc_camiones[camion][4]=tiempo_salida
            COLA.pop(0)
            print("Entra el primer camion de la cola al equipo {} de molienda. Saldrá en {} horas más.".format(equipo_asignado,tiempo_salida))
            
            
            

In [ ]:
def salida_prensado(T,tipo_evento,tiempos_molienda,estados_molienda,tiempos_prensado,estados_prensado,tasa_prensado,merma,tiempos_clarificacion,estados_clarificacion,tasa_clarificacion):
    #rescato y actualizo info del camion
    camion=estados_prensado[tipo_evento[1:]][1] #camion que esta saliendo de prensado Ej "CAMION1"
    tipo_uva=dicc_camiones[camion][0]
    dicc_camiones[camion][1]=dicc_camiones[camion][1]*(1-merma) #merma que se produce en molienda
    kg=dicc_camiones[camion][1]
    calidad=dicc_camiones[camion][2]
    dicc_camiones[camion][3]=dicc_camiones[camion][3]+T #contador del tiempo que lleva fuera la uva

    #ver si clarificacion esta desocupado
    equipo_asignado=0
    for i in estados_clarificacion:
        if estados_clarificacion[i][0]==0:
            equipo_asignado=i 
            estados_clarificacion[i]=[1,camion]
            tiempo_salida=(kg/tasa_clarificacion)
            tiempos_clarificacion["TSC{}".format(equipo_asignado[1])]=T+tiempo_salida
            dicc_camiones[camion][6]=tiempo_salida
            print("Se asigna al equipo {} de clarificacion. Saldrá en {} horas más.".format(equipo_asignado,tiempo_salida))
            break
    #si no hay ningun equipo prensado desocupado
    if equipo_asignado==0:
        estados_prensado[tipo_evento[1:]][0]=2 
        tiempos_prensado[tipo_evento]=math.inf
        print("No hay ningun equipo de clarificacion desocupado. Se bloquea")
    #revisar bloqueados
    else:
        tiempo_min=math.inf
        equipo_min=0
        #reviso todos los equipos de molienda bloqueados
        for i in estados_molienda:
            if estados_molienda[i][0]==2:
                camion=estados_molienda[i][1] #camion que esta en el equipo bloqueado
                tiempo=dicc_camiones[camion][4] #tiempo en que habia terminado la molienda
                if tiempo<tiempo_min: 
                    tiempo_min=tiempo 
                    equipo_min=i #obtengo el equipo de molienda bloqueado que habia terminado antes
                    
        if tiempo_min==math.inf:
            print("no hay equipos de molienda bloqueados")
            estados_prensado[tipo_evento[1:]]=[0,0] #Ej "P1"=[0,0]
            tiempos_prensado[tipo_evento]=math.inf #Ej "TSP1"=inf
        else:
            camion=estados_molienda[equipo_min][1]
            kg=dicc_camiones[camion][1]
            tiempo_salida=T+(kg/tasa_prensado)
            estados_prensado[tipo_evento[1:]]=[1,camion]
            tiempos_prensado[tipo_evento]=T+tiempo_salida
            dicc_camiones[camion][5]=tiempo_salida
            print("Se desbloquea el equipo {} de molienda y entra al equipo {} de prensado".format(equipo_min,estados_prensado[tipo_evento[1:]]))
            
        

In [ ]:
def salida_clarificacion(T,tipo_evento,tiempos_prensado,estados_prensado,tiempos_clarificacion,estados_clarificacion,tasa_clarificacion,merma):
    #rescato y actualizo info del camion
    camion=estados_clarificacion[tipo_evento[1:]][1] #camion que esta saliendo de clarificacion Ej "CAMION1"
    tipo_uva=dicc_camiones[camion][0]
    dicc_camiones[camion][1]=dicc_camiones[camion][1]*(1-merma) #merma que se produce en clarificacion
    kg=dicc_camiones[camion][1]
    calidad=dicc_camiones[camion][2]
    dicc_camiones[camion][3]=dicc_camiones[camion][3]+T #contador del tiempo que lleva fuera la uva

    #revisar bloqueados
    tiempo_min=math.inf
    equipo_min=0
    #reviso todos los equipos de prensado bloqueados
    for i in estados_prensado:
        if estados_prensado[i][0]==2:
            camion=estados_prensado[i][1] #camion que esta en el equipo bloqueado
            tiempo=dicc_camiones[camion][5] #tiempo en que habia terminado el prensado
            if tiempo<tiempo_min: 
                tiempo_min=tiempo 
                equipo_min=i #obtengo el equipo de prensado bloqueado que habia terminado antes

    if tiempo_min==math.inf:
        print("no hay equipos de prensado bloqueados")
        estados_clarificacion[tipo_evento[1:]]=[0,0] #Ej "P1"=[0,0]
        tiempos_clarificacion[tipo_evento]=math.inf #Ej "TSP1"=inf
    else:
        camion=estados_prensado[equipo_min][1]
        kg=dicc_camiones[camion][1]
        tiempo_salida=T+(kg/tasa_clarificacion)
        estados_clarificacion[tipo_evento[1:]]=[1,camion]
        tiempos_clarificacion[tipo_evento]=T+tiempo_salida
        dicc_camiones[camion][6]=tiempo_salida
        print("Se desbloquea el equipo {} de prensado y entra al equipo {} de clarificacion".format(equipo_min,estados_clarificacion[tipo_evento[1:]]))

        

In [ ]:
#input:
#- dicc_lotes: diccionario de los lotes comprados -> "loteX"=[tipo uva, kg, calidad, tiempo llegada]
#- equipos1: numero de equipos de molienda
#- tasa1: tasa de servicio de molienda (kg/hr)
#- equipos2: numero de equipos de prensado
#- tasa2: tasa de servicio de prensado (kg/hr)
#- equipos3: numero de equipos de clarificacion
#- tasa3: tasa de servicio de clarificacion (kg/hr)

def Simular_procesamiento(dicc_lotes,equipos1,tasa1,merma1,equipos2,tasa2,merma2,equipos3,tasa3,merma3):
    T=0
    dicc_camiones=generar_camiones(dicc_lotes)
    tiempos_molienda,estados_molienda=generar_molienda(equipos1)
    tiempos_prensado,estados_prensado=generar_prensado(equipos2)
    tiempos_clarificacion,estados_clarificacion=generar_clarificacion(equipos3)
    COLA=[]
    tipo_evento,tiempo_evento=prox_evento(tiempos_llegadas,tiempos_molienda,tiempos_prensado,tiempos_clarificacion)
    while tiempo_evento!=math.inf:
        T=tiempo_evento
        #si es llegada
        if tipo_evento[0]=="C":
            llegada(T,tipo_evento,dicc_camiones,tiempos_molienda,estados_molienda,tasa1,COLA)
        #si es salida
        elif 'M' in tipo_evento:
            salida_molienda(T,tipo_evento,dicc_camiones,tiempos_molienda,estados_molienda,tasa1,merma1,tiempos_prensado,estados_prensado,tasa2,COLA) 
        elif 'P' in tipo_evento:
            salida_prensado(T,tipo_evento,tiempos_molienda,estados_molienda,tiempos_prensado,estados_prensado,tasa2,merma2,tiempos_clarificacion,estados_clarificacion,tasa3)    
        elif 'C' in tipo_evento:
            salida_clarificacion(T,tipo_evento,tiempos_prensado,estados_prensado,tiempos_clarificacion,estados_clarificacion,tasa3,merma3)  
        tipo_evento,tiempo_evento=prox_evento(tiempos_llegadas,tiempos_molienda,tiempos_prensado,tiempos_clarificacion)
